#4-9 次元圧縮

## 概要
**次元圧縮は、データの次元を圧縮することによって、データの構造を見やすくしたり、機械学習の計算量を軽減して計算スピードを向上させたりする手法です。**

代表的な手法に、主成分分析（Primary Component Analysis, PCA）、t-SNE法があります。

##主成分分析（PCA）

主成分分析は、相関のある複数の変数を、ばらつきの方向と大きさに着目し、より相関の少ない合成関数に変換して、データの次元を縮約する手法です。

下図に2 次元データの例を示します。このデータのばらつき（＝情報）は右肩上がりの方向が最も大きく見えます。

<img src="https://github.com/t-date/DataScience/blob/master/fig/04_09_03.jpg?raw=true" width="280px">

そこで、上図のように、この右肩上がりの方向の軸を第一主成分軸と定義し、直交する軸を第二主成分軸と定義します。この新しい軸にしたがって回転した図が下図になります。

<img src="https://github.com/t-date/DataScience/blob/master/fig/04_09_04.jpg?raw=true" width="350px">

この新たな軸は、このデータの情報をよく表していることになります。このように、データのばらつきの方向と大きさに着目した分析法が主成分分析です。次元圧縮をする際は、よりばらつきの小さい第二主成分軸方向の情報を削減し、第一主成分軸方向に圧縮します。

<img src="https://github.com/t-date/DataScience/blob/master/fig/04_09_05.jpg?raw=true" width="350px">

こうすることにより、より多くデータの情報を残しつつ次元を削減することができます。このとき、主成分の寄与度を表すものを主成分得点（主成分スコア）、観測変数との相関を表すものを主成分負荷量もしくは因子負荷量といい、主成分分析の結果の評価に用いられます。


###主成分分析の実装(あやめの例)

教師なし学習の例として、アイリスデータの次元を減らしてより簡単に可視化する方法を見てみましょう。

アイリスのデータは4 次元でり、各サンプルは4 つの特徴を持ちます。
データの本質的な特徴を保持したまま適切な低次元表現にできるかが次元削減の課題です。多くの場合、次元の削減はデータを可視化するのに役立ちます。4 次元以上のデータをプロットするより2次元のデータをプロットする方がはるかに簡単です。

ここでは、主成分分析を使用します。これは、高速な線形の次元削減手法です。モデルに対して2 つの要素、つまりデータの2 次元表現を返すよう要求します。


Seabornライブラリを使用して、あやめのデータセットをpandas DataFrame の形式でダウンロードします。

In [0]:
import seaborn as sns
iris = sns.load_dataset('iris')
iris.head()

In [0]:
%matplotlib inline
import seaborn as sns; sns.set()
sns.pairplot(iris, hue='species', height=1.5);

scikit-learnのために、DataFrameから特徴行列と目的配列を抽出します。

In [0]:
X_iris = iris.drop('species', axis=1)
X_iris.shape

In [0]:
y_iris = iris['species']
y_iris.shape

In [0]:
from sklearn.decomposition import PCA # 1.モデルクラスを選択する

model = PCA(n_components=2) # 2. ハイパーパラメータを添えて、インスタンス化する
model.fit(X_iris) # 3.データへの当てはめを行う（yを指定していない点に注意）
X_2D = model.transform(X_iris) # 4. 2 次元のデータに変換する

結果をプロットします。元のirisデータフレームに結果を追加して、Seabornのlmplot を使用すします。

In [0]:
iris['PCA1'] = X_2D[:, 0]
iris['PCA2'] = X_2D[:, 1]
sns.lmplot("PCA1", "PCA2", hue='species', data=iris, fit_reg=False);

PCAアルゴリズムに種のラベルを与えていないにもかかわらず、種がかなりよく分離されていることが2次元表現上でわかります。
このデータセットに対しては、比較的簡単な分類が有効であることを示しています。

###主成分分析の実装(手書き文字の例)

scikit-learnに入っている手書き数字データセット（digitsデータセット）に主成分分析を適用してみましょう。

このデータセットの個々のデータポイントは8×8のグレースケールの0から9までの手書き数字です。

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set() # プロットのスタイルを決める
import numpy as np
from sklearn.datasets import load_digits
digits = load_digits()

fig, axes = plt.subplots(2, 5, figsize=(10, 5),
                         subplot_kw={'xticks':(), 'yticks': ()})
for ax, img in zip(axes.ravel(), digits.images):
  ax.imshow(img)

PCAを使ってこのデータを2次元にして可視化してみましょう。最初の2つの主成分を用い、各点をクラスごとに色分けしています。

In [0]:
from sklearn.decomposition import PCA
# PCAモデルを構築
pca = PCA(n_components=2)
pca.fit(digits.data)
# 数値データを最初の2主成分で変形
digits_pca = pca.transform(digits.data)
colors = ["#476A2A", "#7851B8", "#BD3430", "#4A2D4E", "#875525",
          "#A83683", "#4E655E", "#853541", "#3A3120", "#535D8E"]
plt.figure(figsize=(10, 10))
plt.xlim(digits_pca[:, 0].min(), digits_pca[:, 0].max())
plt.ylim(digits_pca[:, 1].min(), digits_pca[:, 1].max())
for i in range(len(digits.data)):
  # 散布図を数字でプロット
  plt.text(digits_pca[i, 0], digits_pca[i, 1], str(digits.target[i]),
           color = colors[digits.target[i]],
           fontdict={'weight': 'bold', 'size': 9})
plt.xlabel("First principal component")
plt.ylabel("Second principal component")

ここでは、どのクラスがどこにあるかを示すために各点をその数字で表現している。数字0と6と4は最初の2主成分で比較的うまく分離できているが、それでも重なっている。他の数字は大きく重なり合っている。

##t-SNE法


**t-SNE 法は、高次元のデータを、自由度１のt 分布を用いて２次元や３次元の低次元に圧縮する手法です。**

t-SNEの各文字は、それぞれ下記を表します。

*   t:t分布
*   S:確率的（Stochastic）
*   N:隣接（Neighbor）
*   E:埋め込み（Embedding）

この手法で次元圧縮すると、離れたグループはより離れて配置されるため、クラスタリングしやすくなるという特徴があります。一方で、圧縮後の次元が4 次元以上だとうまく働かない場合もあるため、2次元もしくは3 次元が推奨されています。
下図に主成分分析とt-SNE法による次元圧縮結果の比較例を示します。

<img src="https://github.com/t-date/DataScience/blob/master/fig/04_09_06.jpg?raw=true" width="700px">


###t-SNE法の実装(手書き文字の例)

主成分分析で実装した手書き数字データセット（digitsデータセット）に対し、結果を比較するために、t-SNEを適用してみましょう。

t-SNEは新しいデータの変換をサポートしていないため、TSNEクラスにはtransformメソッドがありません。これに代えて、fit_transformメソッドを利用します。このメソッドはモデルを作ると同時に、それを使って変換してデータを返します。

In [0]:
from sklearn.manifold import TSNE
tsne = TSNE(random_state=42)
# fitではなくfit_transformを用いる。TSNEにはtransformメソッドがない
digits_tsne = tsne.fit_transform(digits.data)

In [0]:
plt.figure(figsize=(10, 10))
plt.xlim(digits_tsne[:, 0].min(), digits_tsne[:, 0].max() + 1)
plt.ylim(digits_tsne[:, 1].min(), digits_tsne[:, 1].max() + 1)
for i in range(len(digits.data)):
  # 点ではなく数字をテキストとしてプロットする
  plt.text(digits_tsne[i, 0], digits_tsne[i, 1], str(digits.target[i]),
           color = colors[digits.target[i]],
           fontdict={'weight': 'bold', 'size': 9})
plt.xlabel("t-SNE feature 0")
plt.xlabel("t-SNE feature 1")

t-SNEの結果では、すべてのクラスがかなり明確に分離されています。1と9は少し別れているが、ほとんどのクラスは1つの密な集団にまとまっています。

この方法では、クラスラベルの知識をまったく使っていません。これは完全に教師なし学習なのだ。にも関わらず、データをクラスごとにきれいに分離して2次元に表現する方法を、もとの空間の点の近さだけを使って発見したわけであります。

t-SNEアルゴリズムにはチューニングパラメータがいくつかあるが、デフォルトの設定で大抵はうまく機能します。perplexityやearly_exaggerationをいじってみてもよいが、一般に効果は大きくはありません。